In [2]:
! pip install numpy >> /dev/null
! pip install pandas >> /dev/null
! pip install opencv-python >> /dev/null
! pip install scipy >> /dev/null
! pip install torch >> /dev/null
! pip install matplotlib >> /dev/null
! pip install git+https://github.com/NVlabs/nvdiffrast >> /dev/null

  Running command git clone --filter=blob:none --quiet https://github.com/NVlabs/nvdiffrast /tmp/pip-req-build-fbedh8bz


In [7]:
import os
import numpy as np
import pandas as pd
import cv2
import glob
import matplotlib.pyplot as plt

from tqdm import tqdm
from scipy.spatial.transform import Rotation as R

In [ ]:
INPUT_DIR = "data" # replace with your sequence
ORIGINAL_VIDEONAME = "output/orig.mp4"
UNDISTORTED_VIDEONAME = "output/undist.mp4"
RS_REMOVED_VIDEONAME = "output/nors.mp4"
STABILIZED_VIDEONAME = "output/stable_nors.mp4"

# camera params
FPS = 20

# camera intrinsics
# K = [739.1654756101043, 739.1438452683457, 625.826167006398, 517.3370973594253]
K = np.array([[739.1654756101043, 0.0, 625.826167006398],
              [0.0, 739.1438452683457, 517.3370973594253],
              [0.0, 0.0, 1.0]])

# output camera intrinsics
K_new = K # pominyatu potim

# distortion params
D = np.array([0.019327620961435945, 0.006784242994724914, -0.008658628531456217, 0.0051893686731546585])

T_cam_imu = np.array([[-0.0027687291789002095, -0.9999872674970001, 0.004218884048773266, -0.05688384754602901],
                       [-0.9999356528558058, 0.002814949729190873, 0.010989367855138411, 0.007618902284014501],
                       [-0.011001103879489809, -0.004188185992194848, -0.9999307150055582, -0.042436390295094266],
                       [0.0, 0.0, 0.0, 1.0]])

R_cam_imu = T_cam_imu[:3, :3]


dt_rs = 29.47 * 10 ** -6  # in seconds

# frame dimensions
W, H = 1280, 1024

In [5]:
def frames_to_video(frames, output_video, fps=20, isColor=False):
    """
    Create a video from a list of frames.

    :param frames: list of frames
    :param output_video: output video file name (e.g., 'output.mp4')
    :param fps: frames per second (default=20)
    """
    if len(frames) == 0:
        raise ValueError("No frames provided")

    # Use the first frame to get dimensions
    height, width, *channels = frames[0].shape

    # Define video writer (using mp4v codec)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(output_video, fourcc, fps, (width, height), isColor=isColor)

    for frame in tqdm(frames, desc="Writing frames to video: "):
        video.write(frame)
        

frames_paths = sorted(glob.glob(os.path.join(INPUT_DIR, "cam1/data/*.png")))
print(f"Number of frames in the video: {len(frames_paths)}")

frames = [cv2.imread(frame_path, cv2.IMREAD_GRAYSCALE) for frame_path in frames_paths]
frames_to_video(frames, ORIGINAL_VIDEONAME, FPS)

Number of frames in the video: 808


Writing frames to video: 100%|█████████████████████████████████████████████████████████████████████████████████| 808/808 [00:06<00:00, 133.14it/s]


### Homework 2

Given input video from camera with rolling shutter, implement and apply the following algorithms:

    - radial distortion correction (use cv2.fisheye module)
    - rolling shutter correction
    - video stabilization

Expected output of the homework:

    - Jupyter notebook with working code that we can run in real-time
    - plots of camera orientations in time for unstabilized and stabilized motion [plot individual x/y/z components in axis-angle representation]
    - plots of input/output meshgrids for the video frame, where angular velocity has the biggest norm
    - 4 videos stacked together with their frames side-by-side:
        - original
        - undistorted 
        - undistorted + rolling shutter corrected
        - undistorted + rolling shutter corrected + stabilized

Note:
Dataset used in homework provides two sources of camera orientations: IMU (Gyro) measurements of angular velocity and Motion Capture direct measurements of orientation. You should use IMU (Gyro) measurements and implement angular velocity integration. You can use Motion Capture direct measurements of orientation to verify that your integration is implemented correctly.

Rolling shutter correction algorithm is based on this paper: [Digital Video Stabilization and Rolling Shutter Correction using Gyroscopes](https://graphics.stanford.edu/papers/stabilization/).

Dataset: [Rolling-Shutter Visual-Inertial Odometry Dataset](https://cvg.cit.tum.de/data/datasets/rolling-shutter-dataset)

Calibrated camera params can be found [here](https://cdn3.vision.in.tum.de/rolling/calibration/camchain-calibration-equidistant4_camimu_dataset-calib-imu1.yaml)

![Camera Set-up](https://cvg.cit.tum.de/_media/data/datasets/rolling-shutter-dataset/sensor_axes_rgb.jpg?w=800&tok=fa3d4a)

## Preprocessing IMU data

In [16]:
imu_data = pd.read_csv(f"{INPUT_DIR}/imu0/data.csv", header=None, dtype=np.float64, skiprows=1)
imu_data.columns = ["timestamp", "w_x", "w_y", "w_z", "a_x", "a_y", "a_z"]
gyro_data = imu_data[["timestamp", "w_x", "w_y", "w_z"]].to_numpy()
accel_data = imu_data[["timestamp", "a_x", "a_y", "a_z"]].to_numpy()

In [29]:
def integrate_gyro(gyro):
    dt = np.diff(gyro[:, 0], prepend=gyro[0, 0])
    omega = gyro[:, 1:4]
    rotations = []

    R_curr = R.from_quat([1,0,0,0])
    for w, dt_ in tqdm(zip(omega, dt)):
        angle = np.linalg.norm(w * dt_)
        if angle > 0:
            axis = w / np.linalg.norm(w)
            dR = R.from_rotvec(axis * angle)
        else:
            dR = R.from_quat([1,0,0,0])
        R_curr = R_curr * dR
        rotations.append(R_curr.as_matrix())
    return np.array(rotations)

# def integrate_accel(accel):
#     dt = accel[1:, 0] - accel[:-1, 0]
#     dt = np.concatenate(([dt[0]], dt))
#     a = accel[:, 1:4]
#     v = np.cumsum(a * dt[:, np.newaxis], axis=0)
#     p = np.cumsum(v * dt[:, np.newaxis], axis=0)
#     return v, p

In [30]:
rotaions = integrate_gyro(gyro_data)

7647it [00:00, 52891.52it/s]


In [19]:
IMU_fs = 200
imu_to_fps_ratio = IMU_fs // FPS
rotaions_fps = rotaions[::imu_to_fps_ratio]

In [31]:
print(rotaions_fps)

[[[ 1.          0.          0.        ]
  [ 0.          1.          0.        ]
  [ 0.          0.          1.        ]]

 [[ 0.42585699 -0.16231187 -0.89011273]
  [ 0.49249711  0.86685164  0.07755537]
  [ 0.75900753 -0.47140544  0.44909296]]

 [[ 0.99996988  0.00695554 -0.00344405]
  [-0.00148033 -0.26467306 -0.96433707]
  [-0.00761903  0.96431312 -0.2646548 ]]

 ...

 [[-0.01978024 -0.97477925  0.22229295]
  [ 0.25674921 -0.21983274 -0.94114473]
  [ 0.96627563  0.03845747  0.25462214]]

 [[-0.30856201 -0.87158569 -0.38095651]
  [-0.24700936  0.46018055 -0.85277209]
  [ 0.91857273 -0.16903325 -0.35728406]]

 [[ 0.04116155  0.90461274 -0.42424228]
  [ 0.95303215 -0.16306041 -0.25522738]
  [-0.30005906 -0.39381098 -0.86883685]]]


## Undistort frames

In [ ]:
undistorted_frames = [cv2.fisheye.undistortImage(image, K, D, None, K) for image in tqdm(frames)]

In [ ]:
frames_to_video(undistorted_frames, UNDISTORTED_VIDEONAME, FPS)